<a href="https://colab.research.google.com/github/Shri-Aakash/tensorflow-1-public/blob/main/C1W3_Assignment_Completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import required libraries

In [1]:
!nvidia-smi

Sat Jul 13 14:47:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

### Import MNIST dataset from keras

In [3]:
(train_images,train_labels),(test_images,test_labels)=keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
print(f"Training data size:{train_images.shape}")
print(f"Testing data size:{test_images.shape}")

Training data size:(60000, 28, 28)
Testing data size:(10000, 28, 28)


In [5]:
train_images[0].shape

(28, 28)

##Preprocessing the data

In [6]:
def reshape_and_normalize(images):
  ##Reshape the array
  images=np.expand_dims(images,axis=-1)

  ##Normalize pixel values:
  images=np.divide(images,255)
  return images

In [7]:
(train_images,train_labels),(test_images,test_labels)=keras.datasets.mnist.load_data()

train_images=reshape_and_normalize(train_images)
test_images=reshape_and_normalize(test_images)
print(f"Maximum pixel value after normalization: {np.max(train_images)}\n")
print(f"Shape of training set after reshaping: {train_images.shape}\n")
print(f"Shape of one image after reshaping: {train_images[0].shape}")

Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


### Defnining callback for training

In [8]:
class MyCallback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if logs.get('accuracy') is not None and logs['accuracy']>=0.995:
      print("\nnReached 99.5% accuracy so cancelling training!")

      self.model.stop_training=True


In [9]:
def convolutional_model():
  model=tf.keras.Sequential(layers=[
      keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation=tf.nn.relu,input_shape=(28,28,1)),
      keras.layers.MaxPooling2D(pool_size=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(units=512,activation=tf.nn.relu),
      keras.layers.Dense(units=10,activation=tf.nn.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

In [10]:
model=convolutional_model()
callback=MyCallback()
model.fit(train_images,train_labels,epochs=10,callbacks=[callback])

Epoch 1/10
1875/1875 [==============================] - 11s 4ms/step - loss: 0.1137 - accuracy: 0.9646
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0390 - accuracy: 0.9877
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0206 - accuracy: 0.9933
Epoch 4/10
1866/1875 [============================>.] - ETA: 0s - loss: 0.0140 - accuracy: 0.9951
nReached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0141 - accuracy: 0.9951


In [11]:
model.evaluate(test_images,test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0534 - accuracy: 0.9838


[0.0534362718462944, 0.9837999939918518]

## Final model accuracy:
### On training data  : 99.5%
### On testing  data  : 98.4%